In [1]:
import numpy as np
import pandas as pd


from mtist import mtist_utils as mu
from mtist import infer_mtist as im

Plan - 

* For each ground truth, run random inference 10,000 times. Then put 'em all together.

In [2]:
meta = (
    pd.read_csv("mtist_datasets/mtist_metadata.csv")
    .drop(columns=["Unnamed: 0"])
    .set_index("did")
)

rng = np.random.default_rng(11206)

mu.GLOBALS.GT_DIR = "ground_truths"
aijs, _ = mu.load_ground_truths(mu.GLOBALS.GT_DIR)

gts = list(aijs.keys())

In [3]:
# Create a null model

null = {
    gt: 
        [
            im.calculate_es_score(
                aijs[gt],
                rng.uniform(low=-3, high=3, size=aijs[gt].shape)
            )
            for _ in range(10000)
        ]
    for gt in gts
}

null_df = pd.DataFrame(null)
null_df.to_csv('null_models/null_df.csv')

(
    null_df[meta.query("n_species == 3")["ground_truth"].unique()]
    .melt()
    .rename(columns={"value": "3-sp_null"})
    .to_csv("null_models/null_df_3sp.csv")
)

(
    null_df[meta.query("n_species == 10")["ground_truth"].unique()]
    .melt()
    .rename(columns={"value": "10-sp_null"})
    .to_csv("null_models/null_df_10sp.csv")
)

(
    null_df[meta.query("n_species == 100")["ground_truth"].unique()]
    .melt()
    .rename(columns={"value": "100-sp_null"})
    .to_csv("null_models/null_df_100sp.csv")
)

In [17]:
# Create a null model

gt = "100_sp_gt"

strong_th = np.percentile(
    np.abs(pd.DataFrame(aijs[gt]).replace(0, np.nan).melt().dropna()["value"]),
    90,
)

truth_strong = pd.DataFrame(aijs[gt])
truth_strong = truth_strong[np.abs(truth_strong) > strong_th].fillna(0).values

null_strong = {
    gt: [
        im.calculate_es_score(
            truth_strong, rng.uniform(low=-3, high=3, size=(100, 100))
        )
        for _ in range(10000)
    ]
}

strong_df = (
    pd.Series(null_strong[gt])
    .to_frame("100-sp_null")
    .assign(variable=gt)
)

strong_df = strong_df[strong_df.columns[::-1]]

strong_df.to_csv("null_models/null_df_100sp_strong.csv")